# Querying Data from `FTA Bus Award StoryMap`

Currently FTA uses ArcGIS storymap to display their bus award data. Is is possible to extract this data using their REST server.
Extracting bus award data from a server will be the most reliable way to see the data.

In [1]:
import geopandas as gpd
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [2]:
import _bus_cost_utils
from calitp_data_analysis.sql import to_snakecase

# Get geojson from ArcGIS Rest Server 

https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query
https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query


# Steps to Query Rest Server

1. Set `Where:` field to `1=1`
2. Set `Out Field:` field to `*`
3. Set `Format:` to `Geojson`
4. Click `Query(GET)`
5. Copy paste given URL 

In [3]:
fy24 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

fy23 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="


fy_24_data = to_snakecase(gpd.read_file(fy24))
fy_23_data = to_snakecase(gpd.read_file(fy23))

display(
    fy_24_data.shape,
    type(fy_24_data),
    fy_24_data.info(),
    fy_23_data.shape,
    type(fy_23_data),
    fy_23_data.info(),
)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   application_number_       117 non-null    object  
 1   agency_name               117 non-null    object  
 2   low_no_bbf                117 non-null    object  
 3   project_description       117 non-null    object  
 4   number_of_buses_          117 non-null    int64   
 5   funding_amount            116 non-null    float64 
 6   project_title             117 non-null    object  
 7   project_type              117 non-null    object  
 8   display_type              117 non-null    object  
 9   overall_propulsion_type   117 non-null    object  
 10  specific_propulsion_type  117 non-null    object  
 11  areas_served              117 non-null    object  
 12  congressional_districts   117 non-null    object  
 13  address_line_1            117 non-null    

(117, 22)

geopandas.geodataframe.GeoDataFrame

None

(130, 25)

geopandas.geodataframe.GeoDataFrame

None

# Explore FY23 data

In [4]:
fy_23_data = fy_23_data.assign(
    extracted_prop_type=fy_23_data["project_description"].apply(
        _bus_cost_utils.new_prop_finder
    )
)

In [5]:
display(fy_23_data.columns, fy_23_data.head(2))

Index(['application_number', 'fta_region', 'project_sponsor', 'project_title',
       'project_description', 'original_statement_of_work', 'funding',
       'number_of_buses', 'traditional_buses', 'low_emission_buses',
       'zero_emission_buses', 'project_type', 'display_type', 'areas_served',
       'congressional_districts', 'propulsion_type', 'address1', 'address2',
       'city', 'state', 'zip', 'x', 'y', 'objectid', 'geometry',
       'extracted_prop_type'],
      dtype='object')

,application_number,fta_region,project_sponsor,project_title,project_description,original_statement_of_work,funding,number_of_buses,traditional_buses,low_emission_buses,zero_emission_buses,project_type,display_type,areas_served,congressional_districts,propulsion_type,address1,address2,city,state,zip,x,y,objectid,geometry,extracted_prop_type
0,BUSFAC-2023-3691,8,Utah Department of Transportation on behalf of Park City Transit,"Park City's Comprehensive Bus Stop Redesign and Rehabilitation: Increasing Efficiency, Improving ADA Compliance and Preparing for the Expansion of Affordable Housing Opportunities.","The Utah Department of Transportation will receive funding on behalf of Park City Transit to improve more than 72 bus stops, including rider amenities. The project will provide a safer, more appealing interface with riders, particularly in areas with historically disadvantaged communities as well as seasonal visitors.","Through this project, Park City Transit will engage in the design, construction, and installation of improved passenger amenities at 72 individual bus stops within its service area. Of particular emphasis will be the provision of 12 upgraded bus stop sites serving a new affordable housing community of 500 units as well as enhancements to existing SR-248 corridor stops in areas previously identified as historically disadvantaged populations with an elevated degree of transit dependency.","$7,393,183",0,0,0,0,Facility,Zero Facility,Rural,UT-001,Zero / Traditional,4501 Constitution Blvd,NaN,Taylorsville,UT,84129,-111.956609,40.671120,1,POINT (-111.95661 40.67112),not specified
1,BUSFAC-2023-3715,9,Northern Arizona Intergovernmental Public Transportation Authority,"Maintenance Facility Improvements: Essential project to reach community goals for climate action, racial equity, transition to electric buses, and improve mechanic safety and efficiency.","The Northern Arizona Intergovernmental Public Transportation Authority (Mountain Line) will receive funding to build a maintenance facility at the agency's headquarters. This building will house the entire bus fleet, improve maintenance and create jobs in zero-emission technology as it strives to better serve the Flagstaff community.","Funding supports the construction of the maintenance facility at our headquarters. Mountain Line has received $1,600,000 in a competitive FTA Arizona 5339/5307 grant for the design of this facility. The existing facility is not designed for 60 buses, only fitting half the bus inside and is not tall enough to safely perform work on batteries nor has the required electric outlets for electric buses. This project improves fleet reliability by reducing mechanic down time, create good-paying jobs in zero emission technology, modernize our campus and enable Mountain Line to meet growth assumptions that connect communities. Mountain Line completes all maintenance in-house and has high standards to maintain a state of good repair for all fleet, as demonstrated by the fact that Mountain Line has never had a maintenance finding in an FTA triennial review. This project takes a proactive approach to addressing maintenance issues to maintain our high standards for state of good repair.","$16,358,000",0,0,0,0,Facility,Zero Facility,Small Urban,AZ-002,Zero / Low,3773 N Kaspar Drive,NaN,Flagstaff,AZ,86004,-111.592407,35.218441,2,POINT (-111.59241 35.21844),zero-emission bus (not specified)


In [6]:
# clean the funding columns to remove $ and ,
fy_23_data["funding"] = fy_23_data["funding"].str.replace("$", "").str.replace(",", "").astype("int64")

fy_23_data["zero_emission_buses"] = fy_23_data["zero_emission_buses"].astype("int64")

/tmp/ipykernel_569/3907989413.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fy_23_data["funding"] = fy_23_data["funding"].str.replace("$", "").str.replace(",", "").astype("int64")


In [7]:
fy_23_data["propulsion_type"].value_counts()

Zero                        54
Low                         43
Traditional                 17
Zero / Traditional           5
Zero / Low                   3
Low / Traditional            3
Other                        2
Zero / Low / Traditional     2
Combined                     1
Name: propulsion_type, dtype: int64

In [8]:
fy_23_data["propulsion_type"].value_counts()

Zero                        54
Low                         43
Traditional                 17
Zero / Traditional           5
Zero / Low                   3
Low / Traditional            3
Other                        2
Zero / Low / Traditional     2
Combined                     1
Name: propulsion_type, dtype: int64

In [9]:
fy_23_data["project_sponsor"].value_counts().head()

New Mexico Department of Transportation on behalf of North Central Regional Transit District    2
State of California on behalf of Kern Regional Transit                                          2
Regional Public Transportation Authority (Valley Metro)                                         1
Metropolitan Transit Authority of Harris County (METRO)                                         1
State of California on behalf of Glenn County Transportation Commission                         1
Name: project_sponsor, dtype: int64

In [10]:
# filtering down fy23 data to just ZEB rows (by FTA project type and proulsion type)
col_list = [
    "application_number",
    "project_sponsor",
    "project_description",
    "original_statement_of_work",
    "project_type",
    "funding",
    "number_of_buses",
    "zero_emission_buses",
    "propulsion_type",
    "extracted_prop_type",
]

fy_23_zeb_bus = fy_23_data[
    (fy_23_data["project_type"] == "Bus")
    & (fy_23_data["propulsion_type"] == "Zero")
    # (fy_23_data["low_emission_buses"] == 0) &
    # (fy_23_data["zero_emission_buses"] > 0)
][col_list]

In [12]:
display(fy_23_zeb_bus.shape, fy_23_zeb_bus.dtypes, fy_23_zeb_bus.head(2))

(4, 10)

application_number            object
project_sponsor               object
project_description           object
original_statement_of_work    object
project_type                  object
funding                        int64
number_of_buses               object
zero_emission_buses            int64
propulsion_type               object
extracted_prop_type           object
dtype: object

,application_number,project_sponsor,project_description,original_statement_of_work,project_type,funding,number_of_buses,zero_emission_buses,propulsion_type,extracted_prop_type
49,LOWNO-2023-3705,Comanche Nation,Comanche Nation Transit will receive funds to buy battery-electric vehicles and charging equipment and train workers in the new technology. The no-emission vehicles will help improve air quality and fill transportation gaps for residents of Kiowa Comanche Apache tribal lands.,"Comanche Nation Transit is submitting a proposal for the Low-No Emissions Grant. Funding requested is $300,000. The funding will go towards purchasing no emissions vehicles as well as purchasing the infrastructure to support the vehicles. There will also be a workforce development plan to prepare our staff and offer them more opportunity to expand as employees.",Bus,300000,2,2,Zero,BEB
112,LowNo-2023-4146,King County Metro Transit,"King County Metro Transit will receive funding to buy battery-electric buses and charging equipment and train workers to maintain the electric fleet. The project will convert 27 bus routes that serve areas with higher numbers of people of low income as well as expand Metro’s apprenticeship program, including promoting transit careers for residents in underserved communities.","Purchase thirty 40-foot battery electric buses (BEBs) to replace thirty 60 hybrid diesel/electric buses that are beyond their FTA defined useful life benchmark. The buses would support converting 27 routes that operate out of Metros South Base Campus to full zero emissions operations. These routes serve areas which have a higher proportion of Black , Indigenous, and People of Color, and low-income populations than the rest of the County. See Attachment 1b.The project also supports Metros Workforce and Pathways Development programs including: 1) Expands Metros 48-month registered apprenticeship programs (four new apprentices). 2) Expands supportive services to help purchase tools and provide dependent care (ten apprentices). 3) Provides inclusive community outreach events to promote career pathways targeting residents in underserved communities. 4) Includes train-the-trainer classes (15) for Metros Battery Electric Bus Training Academy to support advanced workforce training.",Bus,33552634,30 (BEBs),30,Zero,BEB


# Explore FY 24 data

overall, not as grandular as the FY 23 data. FY24 is missing some ZEB bus counts, original SOW

In [13]:
fy_24_data = fy_24_data.assign(
    extracted_prop_type=fy_24_data["project_description"].apply(
        _bus_cost_utils.new_prop_finder
    )
)

In [14]:
list(fy_24_data.columns)

['application_number_',
 'agency_name',
 'low_no_bbf',
 'project_description',
 'number_of_buses_',
 'funding_amount',
 'project_title',
 'project_type',
 'display_type',
 'overall_propulsion_type',
 'specific_propulsion_type',
 'areas_served',
 'congressional_districts',
 'address_line_1',
 'city',
 'state',
 'additional_zip',
 'zip_code',
 'x',
 'y',
 'fid2',
 'geometry',
 'extracted_prop_type']

In [15]:
fy_24_data.head(2)

,application_number_,agency_name,low_no_bbf,project_description,number_of_buses_,funding_amount,project_title,project_type,display_type,overall_propulsion_type,specific_propulsion_type,areas_served,congressional_districts,address_line_1,city,state,additional_zip,zip_code,x,y,fid2,geometry,extracted_prop_type
0,BUSFAC-2024-4750,"Alaska Department of Transportation, on behalf of City and Borough of Juneau, Capital Transit",BBF,"Alaska Department of Transportation, on behalf of Capital Transit, will receive funding to buy new electric buses, install charging equipment, and initiate a workforce development program. The project will improve service, reliability and air quality throughout the city and borough of Juneau in Alaska’s Southeast panhandle.",6,11855112.0,Mitigating Overcrowding and Enhancing Sustainability in Alaskas Capital City,Vehicle | Facility | Chargers,Zero Bus and Facility,Zero,Battery electric,Rural,AK-001,PO Box 112500,Juneau,AK,2500,99811,-134.41630,58.29971,1,POINT (-134.41631 58.29971),electric (not specified)
1,BUSFAC-2024-5061,City of Montgomery / The M Public Transit System,BBF,"The city of Montgomery's The M transit system will receive funding to buy battery electric buses, charging equipment and to initiate a workforce training program. The project will ensure continued service reliability, maintain a state of good repair and, by improving air quality, advance environmental justice in the Montgomery community.",8,16941377.0,City of Montgomery / The M Public Transit System Electric Bus Program,Vehicle | Chargers,Zero Bus and Facility,Zero,Battery electric,Large Urban,AL-002 | AL-007,103 N. Perry Street,Montgomery,AL,3728,36104,-86.30811,32.37930,2,POINT (-86.30811 32.37930),BEB


In [16]:
display(
    fy_24_data["specific_propulsion_type"].value_counts(),
    fy_24_data["project_type"].value_counts(),
    fy_24_data["extracted_prop_type"].value_counts(),
)

Battery electric                                            38
Diesel-electric hybrid                                      14
N/A                                                         12
CNG                                                         11
Gasoline                                                     9
Hydrogen fuel cell                                           8
Diesel                                                       6
Propane                                                      4
Diesel | Gasoline                                            3
Battery electric | Hydrogen fuel cell                        2
Diesel-electric hybrid | Hydrogen fuel cell                  2
Battery electric | Diesel-electric hybrid | Propane          1
CNG | Gasoline                                               1
CNG | Diesel | Diesel-electric hybrid | Gasoline | Other     1
CNG | Diesel-electric hybrid                                 1
Battery electric | Propane                             

Vehicle                           40
Vehicle | Chargers                15
Facility                          15
Vehicle | Facility | Chargers     10
Vehicle | Facility                 7
Vehicle | Equipment                6
Vehicle | Facility | Equipment     5
Vehicle | Facility                 5
Facility | Chargers                4
Facility | Equipment               3
Vehicle | Chargers | Equipment     3
Facility                           2
Vehicle                            2
Name: project_type, dtype: int64

not specified                        31
BEB                                  22
low emission (hybrid)                18
electric (not specified)             13
FCEB                                 12
CNG                                   9
zero-emission bus (not specified)     8
low emission (propane)                4
Name: extracted_prop_type, dtype: int64

In [17]:
# filtering FY24 data to just show ZEB.

col_list_24 = [
    "application_number_",
    "agency_name",
    "project_description",
    "project_type",
    "funding_amount",
    "number_of_buses_",
    "overall_propulsion_type",
    "specific_propulsion_type",
    "extracted_prop_type",
]

project_val = ["Vehicle ", "Vehicle"]

prop_vals = [
    "Battery electric",
    "Hydrogen fuel cell",
    "Battery electric | Hydrogen fuel cell",
    "Battery electric ",
]

fy_24_zeb_bus = fy_24_data[col_list_24][
    (fy_24_data["project_type"].isin(project_val))
    & (fy_24_data["specific_propulsion_type"].isin(prop_vals))
]

In [18]:
display(
    fy_24_zeb_bus.shape,
    fy_24_zeb_bus.columns,
)

(9, 9)

Index(['application_number_', 'agency_name', 'project_description',
       'project_type', 'funding_amount', 'number_of_buses_',
       'overall_propulsion_type', 'specific_propulsion_type',
       'extracted_prop_type'],
      dtype='object')

In [19]:
fy_24_zeb_bus

,application_number_,agency_name,project_description,project_type,funding_amount,number_of_buses_,overall_propulsion_type,specific_propulsion_type,extracted_prop_type
8,BUSFAC-2024-4738,"California Department of Transportation, on behalf of Morongo Basin Transit Authority","The California Department of Transportation, on behalf of the Morongo Basin Transit Authority, will receive funding to buy a battery-electric bus to replace an older vehicle. This project will improve air quality, safety, reliability and state of good repair in the San Bernadino County service area.",Vehicle,131168.0,1,Zero,Battery electric,BEB
9,BUSFAC-2024-4564,Santa Barbara Metropolitan Transit District,"The Santa Barbara Metropolitan Transportation District will receive funding to buy battery-electric buses and chargers to replace older diesel buses, which will improve service reliability and reduce air pollution.",Vehicle,2894131.0,2,Zero,Battery electric,electric (not specified)
31,LOWNO-2024-4943,Escambia County Board of County Commissioners,"Escambia County Area Transit (ECAT) will receive funding to buy battery electric buses and install a charging station, part of ECAT's plan to transition to a fully zero-emission fleet by 2035. The agency will prioritize deployment to areas with a higher percentage of underrepresented populations that rely heavily on transit and have been disproportionately impacted by pollution.",Vehicle,21272962.0,13,Zero,Battery electric,BEB
49,LOWNO-2024-4643,Greater Peoria Mass Transit District,"Greater Peoria Mass Transit District will receive funding to buy battery-electric buses to replace diesel-engine vehicles. The project will lower emissions, increase safety and provide more reliable transportation to underserved communities in Peoria and surrounding areas.",Vehicle,14415095.0,10,Zero,Battery electric,electric (not specified)
54,BUSFAC-2024-5032,Transit Authority of River City (TARC),"Transit Authority of River City (TARC) will receive funding to buy zero-emission battery-electric buses to replace buses that have exceeded their useful life. TARC will improve its fleet's state of good repair and provide more environmentally friendly transportation for riders to jobs, school, healthcare and more.",Vehicle,3643825.0,3,Zero,Battery electric,BEB
57,LOWNO-2024-4949,Massachusetts Bay Transportation Authority,The Massachusetts Bay Transportation Authority will receive funding to buy new battery electric buses to replace older buses that have exceeded their useful life and additional funds to train workers on the new technology. The zero-emission buses will significantly reduce greenhouse gas emissions and harmful pollutants.,Vehicle,40000000.0,33,Zero,Battery electric,BEB
95,LOWNO-2024-4952,Tri-County Metropolitan Transportation District of Oregon (TriMet),"TriMet will receive funding to buy new hydrogen fuel cell buses to replace older diesel buses that are past their useful life, a new portable fueling station and to initiate facility improvements to accommodate the new hydrogen vehicles. The project will improve service, reliability and provide essential connections to jobs in services in the Portland metro area.",Vehicle,39000000.0,14,Zero,Hydrogen fuel cell,FCEB
106,LOWNO-2024-4980,Utah Transit Authority,"Utah Transit Authority will receive funding to replace older diesel buses with new battery-electric buses, which will be housed at their Meadowbrook facility. This project will reduce harmful emissions, while serving disadvantaged communities within Salt Lake County.",Vehicle,18112632.0,15,Zero,Battery electric,electric (not specified)
114,LOWNO-2024-4946,Chelan Douglas Public Transportation Benefit Area (dba Link Transit),"Link Transit will receive funding to buy battery electric buses, replacing gas-powered buses that have exceeded their useful life. The new buses will be more reliable and easier to maintain, lower fueling costs, reduce air pollution, and improve service throughout North Central Washington.",Vehicle,4

In [22]:
# fixing individual rows
# this is a bandaid until i update the prop finder function

_bus_cost_utils.col_row_updater(fy_24_zeb_bus,"application_number_", "BUSFAC-2024-4564", "extracted_prop_type", "BEB")
_bus_cost_utils.col_row_updater(fy_24_zeb_bus,"application_number_", "LOWNO-2024-4643", "extracted_prop_type", "BEB")
_bus_cost_utils.col_row_updater(fy_24_zeb_bus,"application_number_", "LOWNO-2024-4980", "extracted_prop_type", "BEB")

In [24]:
fy_24_zeb_bus["extracted_prop_type"].value_counts()

BEB     8
FCEB    1
Name: extracted_prop_type, dtype: int64

# Filtered Dataframes
Filtered to show only ZEB projects for 23 and 24 only. 
Project type is just buses, not facilities, equipment or any combinations including buses

In [25]:
display(
    fy_23_zeb_bus.shape,
    fy_23_zeb_bus.dtypes,
    fy_24_zeb_bus.shape,
    fy_24_zeb_bus.dtypes,
)

# 'original_statement_of_work' and 'zero_emission_buses' is not on FY24
# 'specific_propulsion_type' is not in fy23

(4, 10)

application_number            object
project_sponsor               object
project_description           object
original_statement_of_work    object
project_type                  object
funding                        int64
number_of_buses               object
zero_emission_buses            int64
propulsion_type               object
extracted_prop_type           object
dtype: object

(9, 9)

application_number_          object
agency_name                  object
project_description          object
project_type                 object
funding_amount              float64
number_of_buses_              int64
overall_propulsion_type      object
specific_propulsion_type     object
extracted_prop_type          object
dtype: object

In [ ]:
col_dict = {
    "application_number_":"application_number",
    "funding_amount":"funding",
    "zero_emission_buses":"number_of_buses",
    "number_of_buses_":"number_of_buses",
    "project_sponsor":"agency_name",
    "overall_propulsion_type":"propulsion_type",
}

In [ ]:
fy_23_zeb_bus

In [ ]:
fy_24_zeb_bus

In [43]:
test_concat = pd.concat(
    [
        fy_23_zeb_bus[
            [
                "application_number",
        "extracted_prop_type",
        "funding",
        "zero_emission_buses",
        "project_description",
        "project_sponsor",
        "project_type",
        "propulsion_type",
            ]
        ],
        fy_24_zeb_bus[
            [
                "application_number_",
        "extracted_prop_type",
        "funding_amount",
        "number_of_buses_",
        "project_description",
        "agency_name",
        "project_type",
        "overall_propulsion_type"
            ]
        ]
    ]
)



,application_number,extracted_prop_type,funding,zero_emission_buses,project_description,project_sponsor,project_type,propulsion_type,application_number_,funding_amount,number_of_buses_,agency_name,overall_propulsion_type
49,LOWNO-2023-3705,BEB,300000.0,2.0,Comanche Nation Transit will receive funds to buy battery-electric vehicles and charging equipment and train workers in the new technology. The no-emission vehicles will help improve air quality and fill transportation gaps for residents of Kiowa Comanche Apache tribal lands.,Comanche Nation,Bus,Zero,NaN,NaN,NaN,NaN,NaN
112,LowNo-2023-4146,BEB,33552634.0,30.0,"King County Metro Transit will receive funding to buy battery-electric buses and charging equipment and train workers to maintain the electric fleet. The project will convert 27 bus routes that serve areas with higher numbers of people of low income as well as expand Metro’s apprenticeship program, including promoting transit careers for residents in underserved communities.",King County Metro Transit,Bus,Zero,NaN,NaN,NaN,NaN,NaN
123,LowNo-2023-4175,BEB,10000000.0,8.0,"The Metropolitan Bus Authority will receive funding to buy new battery-electric buses, which they will maintain using a solar power charging station that was previously supported with federal funds. The buses will operate on high-visibility routes, reduce greenhouse gas emissions, improve reliability and lessen dependence on external energy sources.",AutoRidad Metropolitana De Autobuses (PRMBA),Bus,Zero,NaN,NaN,NaN,NaN,NaN
128,LowNo-2023-4222,FCEB,29330243.0,23.0,"The North County Transit District will receive funding to buy hydrogen fuel-cell electric buses to replace older buses and to develop a workforce development program in partnership with a local college. This project will help reduce greenhouse gas emissions and improve air quality, health, and transit operations in northern San Diego County.",North County Transit District (NCTD),Bus,Zero,NaN,NaN,NaN,NaN,NaN
8,NaN,BEB,NaN,NaN,"The California Department of Transportation, on behalf of the Morongo Basin Transit Authority, will receive funding to buy a battery-electric bus to replace an older vehicle. This project will improve air quality, safety, reliability and state of good repair in the San Bernadino County service area.",NaN,Vehicle,NaN,BUSFAC-2024-4738,131168.0,1.0,"California Department of Transportation, on behalf of Morongo Basin Transit Authority",Zero
9,NaN,BEB,NaN,NaN,"The Santa Barbara Metropolitan Transportation District will receive funding to buy battery-electric buses and chargers to replace older diesel buses, which will improve service reliability and reduce air pollution.",NaN,Vehicle,NaN,BUSFAC-2024-4564,2894131.0,2.0,Santa Barbara Metropolitan Transit District,Zero
31,NaN,BEB,NaN,NaN,"Escambia County Area Transit (ECAT) will receive funding to buy battery electric buses and install a charging station, part of ECAT's plan to transition to a fully zero-emission fleet by 2035. The agency will prioritize deployment to areas with a higher percentage of underrepresented populations that rely heavily on transit and have been disproportionately impacted by pollution.",NaN,Vehicle,NaN,LOWNO-2024-4943,21272962.0,13.0,Escambia County Board of County Commissioners,Zero
49,NaN,BEB,NaN,NaN,"Greater Peoria Mass Transit District will receive funding to buy battery-electric buses to replace diesel-engine vehicles. The project will lower emissions, increase safety and provide more reliable transportation to underserved communities in Peoria and surrounding areas.",NaN,Vehicle,NaN,LOWNO-2024-4643,14415095.0,10.0,Greater Peoria Mass Transit District,Zero
54,NaN,BEB,NaN,NaN,"Transit Authority of River City (TARC) will receive funding to buy zero-emission battery-electric buses to replace buses that have exceeded their useful life. TARC will improve its fleet's state of good repair and provide more environmentally friendly transportation for riders to jobs, school, healthcare and more.",NaN,Vehicle

In [44]:
display(
    test_concat.shape,
    test_concat.columns
)

(13, 13)

Index(['application_number', 'extracted_prop_type', 'funding',
       'zero_emission_buses', 'project_description', 'project_sponsor',
       'project_type', 'propulsion_type', 'application_number_',
       'funding_amount', 'number_of_buses_', 'agency_name',
       'overall_propulsion_type'],
      dtype='object')

In [34]:
test_concat

,application_number,project_sponsor,project_description,original_statement_of_work,project_type,funding,number_of_buses,zero_emission_buses,propulsion_type,extracted_prop_type,application_number_,agency_name,funding_amount,number_of_buses_,overall_propulsion_type,specific_propulsion_type
0,LOWNO-2023-3705,Comanche Nation,Comanche Nation Transit will receive funds to buy battery-electric vehicles and charging equipment and train workers in the new technology. The no-emission vehicles will help improve air quality and fill transportation gaps for residents of Kiowa Comanche Apache tribal lands.,"Comanche Nation Transit is submitting a proposal for the Low-No Emissions Grant. Funding requested is $300,000. The funding will go towards purchasing no emissions vehicles as well as purchasing the infrastructure to support the vehicles. There will also be a workforce development plan to prepare our staff and offer them more opportunity to expand as employees.",Bus,300000.0,2,2.0,Zero,BEB,NaN,NaN,NaN,NaN,NaN,NaN
1,LowNo-2023-4146,King County Metro Transit,"King County Metro Transit will receive funding to buy battery-electric buses and charging equipment and train workers to maintain the electric fleet. The project will convert 27 bus routes that serve areas with higher numbers of people of low income as well as expand Metro’s apprenticeship program, including promoting transit careers for residents in underserved communities.","Purchase thirty 40-foot battery electric buses (BEBs) to replace thirty 60 hybrid diesel/electric buses that are beyond their FTA defined useful life benchmark. The buses would support converting 27 routes that operate out of Metros South Base Campus to full zero emissions operations. These routes serve areas which have a higher proportion of Black , Indigenous, and People of Color, and low-income populations than the rest of the County. See Attachment 1b.The project also supports Metros Workforce and Pathways Development programs including: 1) Expands Metros 48-month registered apprenticeship programs (four new apprentices). 2) Expands supportive services to help purchase tools and provide dependent care (ten apprentices). 3) Provides inclusive community outreach events to promote career pathways targeting residents in underserved communities. 4) Includes train-the-trainer classes (15) for Metros Battery Electric Bus Training Academy to support advanced workforce training.",Bus,33552634.0,30 (BEBs),30.0,Zero,BEB,NaN,NaN,NaN,NaN,NaN,NaN
2,LowNo-2023-4175,AutoRidad Metropolitana De Autobuses (PRMBA),"The Metropolitan Bus Authority will receive funding to buy new battery-electric buses, which they will maintain using a solar power charging station that was previously supported with federal funds. The buses will operate on high-visibility routes, reduce greenhouse gas emissions, improve reliability and lessen dependence on external energy sources.","PRIME+ proposes to purchase eight no emission buses to eight replace diesel buses and use an FTA FY 2022 awarded dual solar powered charging station to bring sustainable climate friendly transit to Puerto Rico thus effectuating reliable, efficient and clean transportation, as well as, the use of plentiful solar energy. The buses will be assigned to mainline routes showcasing the project to citizens and visitors. PRIME will reduce: fossil fuel consumption, harmful emissions, dependence on foreign energy sources.",Bus,10000000.0,8 (zero emission buses),8.0,Zero,BEB,NaN,NaN,NaN,NaN,NaN,NaN
3,LowNo-2023-4222,North County Transit District (NCTD),"The North County Transit District will receive funding to buy hydrogen fuel-cell electric buses to replace older buses and to develop a workforce development program in partnership with a local college. This project will help reduce greenhouse gas emissions and improve air quality, health, and transit operations in northern San Diego County.","The Accelerate Clean Transit (ACT) Project will deploy 23 hydrogen fuel-cell electric buse